In [1]:
import cufflinks as cf
from numpy import round
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
%matplotlib inline

In [3]:
# Modo offline
init_notebook_mode(connected=True)
cf.go_offline()

# Análise Exploratória

    Métodos de Pagamento

In [4]:
pagamentos = pd.read_csv('olist_public_dataset_v2_payments.csv')

In [5]:
pagamentos.head()

,order_id,installments,sequential,payment_type,value
0,c3d9e402b6a0fbe2a5f7fc5b41117c38,4,1,credit_card,237.45
1,733d6bf02be72825b2c6f120262b9aa9,1,1,credit_card,165.90
2,b2f6a51bb30b26fbe248117ce5eadcfd,1,1,credit_card,55.95
3,d10b4156a519c3f1bc880f6600d32a59,7,1,credit_card,151.76
4,80606b26965c5ed21e85a085e0667b63,4,1,credit_card,178.11


In [6]:
# Visualizar melhor com 2 casas decimais

round(pagamentos.describe(), 2)

,installments,sequential,value
count,104848.00,104848.00,104848.00
mean,2.96,1.11,153.30
std,2.76,0.79,207.02
min,0.00,1.00,0.00
25%,1.00,1.00,59.85
50%,2.00,1.00,102.48
75%,4.00,1.00,174.86
max,24.00,29.00,13664.08


In [7]:
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104848 entries, 0 to 104847
Data columns (total 5 columns):
order_id        104848 non-null object
installments    104848 non-null int64
sequential      104848 non-null int64
payment_type    104848 non-null object
value           104848 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 3.2+ MB


**Formas de Pagamento mais utilizadas**

In [8]:
formas_pagamento = pagamentos['payment_type'].value_counts()
formas_pagamento

credit_card    77209
boleto         20163
voucher         6110
debit_card      1366
Name: payment_type, dtype: int64

Enquanto o *CREDIT CARD* foi o mais utilizado, ja o *DEBIT CARD* fica por ultimo

In [10]:
# Ver em grafico os valores para cada tipo de pagamento

labels = ['Cartão de Crédito',
          'Boleto',
          'Voucher',
          'Cartão de Débito']

values = formas_pagamento.values

colors = ['#388e3c', '#1976d2',
          '#fbc02d', '#c62828']

# Pizza

trace = go.Pie(labels=labels,
               values=values,
               hoverinfo='value+label',
               textinfo='percent',
               textfont=dict(size=12),
               textposition='outside',
               hole=.4,
               marker=dict(colors=colors,
                           line=dict(color='#ffffff',
                                     width=4)))

iplot([trace])

**Parcelamento com CREDIT CARD**

     Numero de compras com MAIS DE UMA PARCELA

In [12]:
parcelas = pagamentos[(pagamentos['payment_type'] == 'credit_card') &
                     (pagamentos['installments'] > 1)]['order_id'].count()

In [13]:
uma_parcela = pagamentos[(pagamentos['payment_type'] == 'credit_card') &
                              (pagamentos['installments'] <= 1)]['order_id'].count()

In [14]:
print('Parcelados: {}'.format(parcelas))
print('Parcela única: {}'.format(uma_parcela))

Parcelados: 53466
Parcela única: 23743


    O Numero de compradores no CREDIT CARD esta acima de 77Mil, sendo:

* Os 53466 restantes parcelaram a compra em duas ou mais vezes.
* 23743 compraram em parcela unica (à vista);

**Veja o Bar Chart**

In [15]:
x = ['Parcela única',
    'Parcelados']
y = [uma_parcela,
     parcelas]

# Barras

trace = go.Bar(x=x, y=y,
               marker=dict(color=['#C51162',
                                  '#4A148C']))

layout = go.Layout(yaxis=dict(title='Compras'),
                   bargap=.1)

fig = go.Figure(data=[trace],
                layout=layout)
iplot(fig)

In [16]:
cc_valores = pagamentos[(pagamentos['payment_type'] == 'credit_card') &
                    (pagamentos['installments'] > 1)]['value']

print('Valor mínimo parcelado: R${}\n'.format(cc_valores.min()) +
      'Média dos valores parcelados: R${:.2f}'.format(round(cc_valores.mean(), 2)))

Valor mínimo parcelado: R$20.28
Média dos valores parcelados: R$191.40


**Numero de Compras por parcelamento**

As compras podem ser dividas em até 24x.
Normalmente passe de 12 parcelas, Veremos aqui as 10 mais populares. ***Note acima de 6x**

In [17]:
# Numero de compras segundo a quantidade de parcelamento

prch_inst = pagamentos[pagamentos['payment_type'] == 'credit_card']\
                ['installments'].value_counts().head(10)

# Separando os índices e os valores
x = prch_inst.index
y = prch_inst.values

# Gráfico de barras
trace = go.Bar(x=x, y=y,
               marker=dict(color='#00796B'))

layout = go.Layout(xaxis=dict(title='Número de Parcelas'),
                   yaxis=dict(title='Compras'),
                   bargap=.1)

fig = go.Figure(data=[trace],
                layout=layout)
iplot(fig)

# Valor da compra e os tipos de pagamento

Relacionando métodos de pagamento com o valor das compras.

In [19]:
# Separando os valores de compra para cada método de pagamento
boleto = pagamentos[pagamentos['payment_type'] == 'boleto']['value']
credit_card = pagamentos[pagamentos['payment_type'] == 'credit_card']['value']
debit_card = pagamentos[pagamentos['payment_type'] == 'debit_card']['value']
voucher = pagamentos[pagamentos['payment_type'] == 'voucher']['value']

# legendas para os tipos de pagamento que serão mostradas pelos gráficos
labels = ['Boleto',
          'Cartão de Crédito',
          'Cartão de Débito',
          'Voucher']

    valores  em cada categoria apos montar estruturas dos gráficos

In [21]:
# Estrutura do gráfico para a média dos valores de compra

# calculando as médias
avg_valores = [boleto.mean(),
              credit_card.mean(),
              debit_card.mean(),
              voucher.mean()]

# Médias com duas casas decimais
avg_valores = [round(i, 2) for i in avg_valores]

# gráfico
avg_pagamentos = go.Bar(x=labels,
                      y=avg_valores,
                      marker=dict(color='#F57F17'))

layout_avg = go.Layout(title='Média dos Valores de Compra',
                       yaxis=dict(title='Valor em R$'),
                       bargap=.1)

fig_avg = go.Figure(data=[avg_pagamentos],
                    layout=layout_avg)

In [22]:
# Estrutura do gráfico para a MAX dos valores de compra

# Maximo
max_valores = [boleto.max(),
              credit_card.max(),
              debit_card.max(),
              voucher.max()]

# Duas casas decimais
max_valores = [round(i, 2) for i in max_valores]

# gráfico
max_pagamentos = go.Bar(x=labels,
                      y=max_valores,
                      marker=dict(color='#311B92'))

layout_max = go.Layout(title='Valores mais altos',
                       yaxis=dict(title='Valor em R$'),
                       bargap=.1)

fig_max = go.Figure(data=[max_pagamentos],
                    layout=layout_max)

In [23]:
# Estrutura do gráfico com os valores mímimos de compra
# PS: Cartão de crédito e voucher são de no mínimo 1 centavo

min_pagamentos = go.Bar(x=['Boleto',
                         'Cartão de Débito'],
                      y=[boleto.min(),
                         debit_card.min()],
                      marker=dict(color='#880E4F'))

layout_min = go.Layout(title='Compras com os Valores Mais Baixos',
                       yaxis=dict(title='Valor em R$'),
                       bargap=.1)

fig_min = go.Figure(data=[min_pagamentos],
                    layout=layout_min)

In [24]:
# Soma das compras por método de pagamento
total = [boleto.sum(),
         credit_card.sum(),
         debit_card.sum(),
         voucher.sum()]

total_pagamentos = go.Bar(y=labels,
                        x=total,
                        marker=dict(color='#2e7d32'),
                        orientation='h')

layout_total = go.Layout(title='Total das Vendas',
                         xaxis=dict(title='Valor em R$'),
                         bargap=.1,
                         margin=dict(l=110))

fig_total = go.Figure(data=[total_pagamentos],
                      layout=layout_total)

**Visualizar Resultados**

In [25]:
# Visualização gráfica da média dos valores de compra
iplot(fig_avg)

In [26]:
# Visualização gráfica das compras com valores mais altos
iplot(fig_max)

In [27]:
# Visualização gráfica das compras com valores mais baixos
# (boleto e cartão de débito)
iplot(fig_min)

In [28]:
# Visualização gráfica do total das vendas para cada método de pagamento
iplot(fig_total)

**Total das Vendas (Aproximado)**

* Boleto: Reais 2,98 milhões<br> 
* Cartão de crédito: Reais 12,5 milhões
* Cartão de Débito: Reais 175 mil<br> 
* Voucher: Reais 371 mil
*    **Total: Reais 16 milhões**